In [1]:
import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re
import tqdm
import json
from nltk.stem.snowball import RussianStemmer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor


from catboost import Pool
from catboost import CatBoostClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Дмитрий\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Дмитрий\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test = pd.read_parquet('internship_2023/test.parquet')
train = pd.read_parquet('internship_2023/train.parquet')

In [4]:
train.columns

Index(['product_id', 'category_id', 'sale', 'shop_id', 'shop_title', 'rating',
       'text_fields', 'category_name'],
      dtype='object')

In [5]:
test.columns

Index(['product_id', 'sale', 'shop_id', 'shop_title', 'rating', 'text_fields'], dtype='object')

In [6]:
train = train.reset_index()
test = test.reset_index()

In [7]:
train = train[['product_id', 'shop_id', 'text_fields', 'category_name']]
test = test[['product_id', 'shop_id', 'text_fields']]

In [8]:
train.head(5)

,product_id,shop_id,text_fields,category_name
0,325286,9031,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,18305,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
2,1267173,16357,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
3,1416943,34666,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
4,1058275,26389,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


In [9]:
test.head(5)

,product_id,shop_id,text_fields
0,1997646,22758,"{""title"": ""Светодиодная лента Smart led Strip ..."
1,927375,17729,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
2,1921513,54327,"{""title"": ""Проводные наушники с микрофоном jac..."
3,1668662,15000,"{""title"": ""Декоративная табличка \""Правила кух..."
4,1467778,39600,"{""title"": ""Подставка под ложку керамическая, п..."


In [10]:
lecn = LabelEncoder()
train['category_name'] = lecn.fit_transform(train['category_name'])

In [11]:
def get_titles(texts): #из text_filds достать только {'title' : *title*}
    titles = []
    for entry in tqdm.tqdm(texts.values):
        entry = entry.replace("'", "")
        entry = json.loads(entry)
        titles.append(entry['title'])
    return titles

In [12]:
def preprocess_texts(texts):
    stop_words = set(stopwords.words('russian')) #не несущие смысла слова
    regex = re.compile('[^а-яА-Я]') #регулярка чтобы оставить слова
    preprocess_texts = []
    for i in tqdm.tqdm(range(len(texts))):
        text = texts[i].lower() #нижний регистр
        text = regex.sub(' ', text) #удалим все символы кроме a-z
        word_tokens = word_tokenize(text) #разобьем на слова
        filtered_sentence = [w for w in word_tokens if not w in stop_words] #удаляем не несущие смысл слова
        preprocess_texts.append(' '.join(filtered_sentence))

    return preprocess_texts #возвращаем предобработанный текст

In [13]:
def stemming_texts(texts): #у каждого слова отрежем окончания
    st = RussianStemmer()
    stem_text = []
    for text in tqdm.tqdm(texts):
        word_tokens = word_tokenize(text)
        stem_text.append(' '.join([st.stem(x) for x in word_tokens.copy()])) 

    return stem_text


In [14]:
train['text_fields'] = stemming_texts(preprocess_texts(get_titles(train['text_fields'])))

100%|██████████| 91120/91120 [00:22<00:00, 4129.30it/s]


---------------

# Попытка в предобработку для catboost
Тут я обрезаю количество примеров чтобы учился быстрее, </br>
И убираю 4 представителя класса, которые единственные на класс </br>

Не захотел учиться на GPU, ребутило ядро юпитер ноутбука, и я уже не успеваю </br>

Исходя из динамики на kaggle, где обучаться он все таки решился (с теми же настройками): </br>
it0   - test 0.005 </br>
it50  - test 0.12 </br>
it100 - test 0.17 </br>
it150 - test 0.227 </br>
it200 - test 0.273 </br>
it250 - test 0.325 </br>
it299 - test 0.36 </br>

Можно предположить что после тысячной итерации можно было бы блендить модели </br>
Оставлю ожидаемый блендинг, но с catboost(iteratations=5)

________________________________________________

In [22]:
catb_train = train.copy()

In [23]:
un_cat = catb_train['category_name'].unique()

classes_0 = {}
for i in un_cat: #присваиваем каждой уникальной категории : 0
    classes_0[i] = 0

In [24]:
classes = catb_train.to_numpy().copy()

In [25]:
limit = 15
indexes = []
for i in range(len(catb_train)): #ограничим количество из одной категории
    if classes_0[ classes[i][-1] ] < limit:
        classes_0[ classes[i][-1] ] += 1
    else:
        indexes.append(i)

In [26]:
len(indexes) #столько строк классов, которые первысили лимит на один класс

79562

In [27]:
catb_train = catb_train.drop(indexes) #удаляем превысившие лимит

In [28]:
todel = []

for key, value in classes_0.items():
    if value == 1:
        todel.append(key)

In [29]:
todel #одиночные представители класса, под удаление

[253, 262, 208, 91]

In [30]:
for d in todel:
    catb_train = catb_train.drop( catb_train.where(catb_train['category_name'] == d).dropna().index )

In [35]:
catb_train = catb_train.reset_index(drop=True)
#catb_train = catb_train.drop(columns=['index'])

In [36]:
df = catb_train[['shop_id', 'text_fields']].copy()
target = catb_train['category_name']

In [37]:
len(catb_train['category_name'].unique()) #осталось классов

870

In [38]:
df.shape #осталось строк

(11554, 2)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df, target, train_size=0.75, test_size=0.25, random_state=777)

In [40]:
cat_features = ['shop_id']

In [41]:
text_features = ['text_fields']

In [42]:
# pool = Pool(X_train, y_train, cat_features=cat_features, feature_names=list(X_train.columns), text_features=text_features)

In [43]:
train_pool = Pool(data=X_train, label=y_train,
                  cat_features = ['shop_id'],
                  text_features= ['text_fields'])

valid_pool = Pool(data=X_test, label=y_test, 
                  cat_features= ['shop_id'],
                  text_features= ['text_fields'])

In [274]:
# grid = {'iterations': [500, 1000, 2500],
#         'learning_rate': [0.03, 0.05],
#         'depth': [2, 3, 4],
#         'l2_leaf_reg': [1, 2, 3],
#         'random_state':[777],
#         'metric_period':[250],
#         'logging_level':['Silent'],
#         'task_type':['GPU'],
#         'devices':['0'],
#         'eval_metric':['TotalF1']}

In [275]:
# gscb = CatBoostClassifier()
# gsresult = gscb.grid_search(grid, X=pool, plot=True)
# params = gsresult['params'] 

In [93]:

cbc = CatBoostClassifier(iterations=5, learning_rate=0.3, max_depth=5,
                        random_seed=7777, metric_period=5, 
                        task_type='CPU', devices='0', eval_metric='TotalF1') #l2_leaf_reg=1, 

cbc.fit(train_pool, plot=True, eval_set=valid_pool, use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0039694	test: 0.0024728	best: 0.0024728 (0)	total: 2m 28s	remaining: 9m 53s
4:	learn: 0.0165518	test: 0.0076480	best: 0.0076480 (4)	total: 12m 5s	remaining: 0us

bestTest = 0.007648023374
bestIteration = 4



_______________________________

# KNN на скорую руку

In [358]:
dfn = catb_train[['shop_id', 'text_fields']].copy()
target = catb_train['category_name']

In [560]:
ohe = OneHotEncoder()
ohe.fit(train[['shop_id', 'text_fields']]) #фитим энкодер на всем датасете со всеми классами
oheded = ohe.transform(dfn) #применяем на нашу подвыборку

In [361]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(oheded, target, train_size=0.75, test_size=0.25, random_state=7777)

In [385]:
knr = KNeighborsRegressor(n_neighbors=1, weights='uniform', algorithm='auto', n_jobs=4)
knr.fit(X_train1, y_train1)

KNeighborsRegressor(n_jobs=4, n_neighbors=1)

In [386]:
f1_score(y_test1, knr.predict(X_test1).round(), average='weighted')

0.19274809960529546

Тест на всем датасете

In [387]:
full_test = train[['shop_id', 'text_fields']].copy()
full_target = train['category_name']
oheded1 = ohe.transform(full_test)
X_train2, X_test2, y_train2, y_test2 = train_test_split(oheded1, full_target, train_size=0.75, test_size=0.25, random_state=7777)

In [558]:
f1_score(y_test2, knr.predict(X_test2).round(), average='weighted')

0.25657079984421555

# LinearSVC над BOW

In [44]:
def bow(vectorizer, train, test):
    train_bow = vectorizer.fit_transform(train)
    test_bow = vectorizer.transform(test)
    return train_bow, test_bow

In [45]:
vectorizer_tf_idf = TfidfVectorizer(ngram_range=(1, 2))

In [46]:
X_train_tf_idf, X_test_tf_idf = bow(vectorizer_tf_idf, X_train['text_fields'], X_test['text_fields'])

In [47]:
X_train_tf_idf.shape

(8665, 24545)

In [48]:
clf_svc = LinearSVC()

In [49]:
clf_svc.fit(X_train_tf_idf, y_train)

LinearSVC()

In [50]:
print( 'LinearSVC:', f1_score(clf_svc.predict(X_test_tf_idf), y_test, average='weighted') )

LinearSVC: 0.7020832267839171


In [59]:
clf_svc.predict(X_test_tf_idf)

array([522, 823, 786, ..., 634, 569, 547])

___________________________________

# Голосование

#def voting(data, clf, cbc, knn): 

Предобработка для каждой из моделей -> предикт -> </br>
Если один и тот же класс предсказали хотя бы 2 модели - отдаем как ответ </br>
Если предсказания не совпадают, отдаем предпочтение ответу модели с наибольшим f1score

--------------

# Для TG лидерборда

In [ ]:
def create_folds(data, target, num_splits=3):
    if num_splits > 1:
        data.loc[:,'kfold'] = -1
        X = data.drop([target], axis=1)
        y = data[[target]]
        mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
        for fold, (trn_, val_) in enumerate(mskf.split(X, y)):
            data.loc[val_,'kfold'] = fold
    else:
        data.loc[:,'kfold'] = 0
    return data

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [ ]:
N = 5
df = create_folds(train, 'category_name', N)
X = df.drop(columns=['category_name'])
y = df[['category_name', 'kfold']]

In [ ]:
val_ds = pd.DataFrame({'true': y['category_name']})
val_ds['pred'] = -1
for kfold in range(5):
    X_train, X_test = X[X.kfold!=kfold].drop('kfold', axis=1), X[X.kfold==kfold].drop('kfold', axis=1)
    y_train, y_test = y[y.kfold!=kfold].drop('kfold', axis=1), y[y.kfold==kfold].drop('kfold', axis=1)
    print(X_train.shape)
    print(f'--------------------------------{kfold}-fold-------------------------------')
    
    X_train['data'] = stemming_texts(preprocess_texts(get_titles(X_train['text_fields'])))    
    X_test['data'] = stemming_texts(preprocess_texts(get_titles(X_test['text_fields'])))
    X_tr, X_te = bow(vectorizer_tf_idf, X_train['data'], X_test['data'])
    #своя модель
    clf_svc = LinearSVC()
#     #clf = LinearRegression()
    clf_svc.fit(X_tr, y_train['category_name'])

    val_ds.loc[y_test.index, 'pred'] = clf_svc.predict(X_te).astype(int)

In [ ]:
f1_score(val_ds['true'], val_ds['pred'], average='weighted')

_________________________

# Итоговое предсказание

In [51]:
test['text_fields'] = stemming_texts(preprocess_texts(get_titles(test['text_fields'])))

100%|██████████| 16860/16860 [00:04<00:00, 4148.72it/s]


In [52]:
def bow(vectorizer, test):
    test_bow = vectorizer.transform(test)
    return test_bow

In [53]:
X_test_tf_idf = bow(vectorizer_tf_idf, test['text_fields'])

In [54]:
result = pd.DataFrame(test['product_id'])

In [55]:
result['predicted_category_id'] = clf_svc.predict(X_test_tf_idf)

In [56]:
result['predicted_category_id'] = lecn.inverse_transform(result['predicted_category_id'])

In [57]:
result.head()

,product_id,predicted_category_id
0,1997646,Все категории->Товары для дома->Товары для пра...
1,927375,Все категории->Электроника->Смартфоны и телефо...
2,1921513,Все категории->Электроника->Наушники и аудиоте...
3,1668662,Все категории->Товары для дома->Товары для кух...
4,1467778,Все категории->Товары для дома->Товары для кух...


In [58]:
test

,product_id,shop_id,text_fields
0,1997646,22758,светодиодн лент пульт метр
1,927375,17729,стекл пленк керамик матов
2,1921513,54327,проводн наушник микрофон
3,1668662,15000,декоративн табличк прав кухн подставк горяч ра...
4,1467778,39600,подставк ложк керамическ подложк клубник лаван...
...,...,...,...
16855,1914264,8598,жестк диск внутрен твердотел накопител
16856,1310569,27474,браслет оберег рук красн нит сглаз защит сглаз...
16857,978095,23395,кабошон бантик упаковк шт
16858,797547,16764,полк ван углов х х см цвет хром


In [63]:
orig = pd.read_parquet('internship_2023/test.parquet')
orig[['product_id','text_fields']]

,product_id,text_fields
1,1997646,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,"{""title"": ""Подставка под ложку керамическая, п..."
...,...,...
24987,1914264,"{""title"": ""Жесткий диск внутренний SSD KingDia..."
24988,1310569,"{""title"": ""Браслет оберег на руку/красная нить..."
24989,978095,"{""title"": ""Кабошон бантик в упаковке 2 шт"", ""d..."
24992,797547,"{""title"": ""Полка для ванной угловая, 20,5 х 20..."


In [592]:
result.to_parquet('result.parquet', engine='pyarrow')